In [ ]:
from src.projects.fagradalsfjall.common.model_eval import ModelEvalResult, ValidationType
from src.projects.fagradalsfjall.common.model_eval_plot_curves import generate_all_rmse_plots
from src.projects.fagradalsfjall.common.model_eval_plot_simulations import plot_simulations
from src.projects.fagradalsfjall.common.model_repo import ModelRepo
from src.projects.fagradalsfjall.common.paths import get_blog_post_subfolder
from src.projects.fagradalsfjall.common.project_settings import TS_PRIMARY_METRIC_DISPLAY_NAME
from src.tools.datetime import format_timedelta

# 2. Visualization

In [ ]:
# -------------------------------------------------------------------------
#  Output path settings
# -------------------------------------------------------------------------
path_results_mlp_smart = get_blog_post_subfolder(6, "results_3_mlp_smart")
path_results_mlp_overview = get_blog_post_subfolder(6, "results_4_overview")

In [ ]:
# -------------------------------------------------------------------------
#  Load data
# -------------------------------------------------------------------------

# --- load models -------------------------------------
prev_model_eval_results = ModelRepo.load_models(["naive-mean", "naive-constant", "naive-exp-decay", "linear-ar"])

all_mlp_model_eval_results = ModelRepo.load_models(["mlp-rmse-n-1", "mlp-rmse-n-192", "mlp-auc", "mlp"])

# rename
all_mlp_model_eval_results = {
    "mlp-mse-n-1": all_mlp_model_eval_results["mlp-rmse-n-1"],
    "mlp-mse-n-192": all_mlp_model_eval_results["mlp-rmse-n-192"],
    "mlp-auc": all_mlp_model_eval_results["mlp-auc"],
    "mlp": all_mlp_model_eval_results["mlp"],
}

final_mlp_model_eval_results = ModelRepo.load_models(["mlp"])

# --- post-processing ---------------------------------
model_mlp_final = all_mlp_model_eval_results["mlp"].ts_model

In [ ]:
# -------------------------------------------------------------------------
#  Overall grid search 1D results
# -------------------------------------------------------------------------
param_names = ["activation", "loss", "lr_max", "n_epochs", "wd", "dropout", "n_hidden_layers", "layer_width", "p", "n"]

for param_name in param_names:

    fig, ax = (
        model_mlp_final.cv.results.plot_1d(param_names=param_name)
        .set_fig_title(f"Cross-validation - 'mlp-final' - best results for each value of '{param_name}'")
        .set_y_label(TS_PRIMARY_METRIC_DISPLAY_NAME)
        .set_x_label(param_name)
        .create(w=12, h=8)
    )
    fig.savefig(path_results_mlp_smart / f"cv_1d_best_{param_name}.png", dpi=600)

In [ ]:
# -------------------------------------------------------------------------
#  Test set results - RMSE Curves
# -------------------------------------------------------------------------

# merge all evaluation results in single dict for visualization
model_eval_results = prev_model_eval_results | final_mlp_model_eval_results

# create RMSE plots
generate_all_rmse_plots(model_eval_results, path_results_mlp_overview, highlight_models=["mlp-final"])

In [ ]:
# -------------------------------------------------------------------------
#  Test set results - Simulations
# -------------------------------------------------------------------------

# simulations
plot_simulations(model_eval_results, path_results_mlp_overview)

In [ ]:
# -------------------------------------------------------------------------
#  Metric values
# -------------------------------------------------------------------------

df_metrics = ModelEvalResult.all_metric_values_as_df(model_eval_results, ValidationType.TEST)

df_metrics["max_accurate_lead_time_samples"] = df_metrics["max_accurate_lead_time"].copy()
df_metrics["max_accurate_lead_time"] = df_metrics["max_accurate_lead_time"].apply(
    lambda x: format_timedelta(x * 15 * 60)
)

display(df_metrics)